# Backtesting & Strategy Validation

This notebook demonstrates a framework for backtesting the performance of a trained machine learning model on out-of-sample historical data. The process simulates trade execution on a minute-by-minute basis, accounting for realistic costs like the bid-ask spread, and provides a clear analysis of the strategy's profitability.

The key steps are:
1.  **Setup:** Load the trained Keras model, data scalers, and helper functions.
2.  **Data Loading:** Load a dataset that the model has not been trained on.
3.  **Simulation Loop:** Iterate through the dataset minute-by-minute, maintaining a buffer of recent data to feed the model.
4.  **Inference & Trade Logic:** For each minute, preprocess the data, generate a prediction, and execute trades based on a defined strategy.
5.  **Performance Analysis:** Calculate and visualize the portfolio's profit and loss (PnL) over time.

### 1. Setup: Load Model and Dependencies

In [ ]:
import pandas as pd
import numpy as np
import joblib
from collections import deque
from keras.models import load_model
import matplotlib.pyplot as plt

# --- Load key components and helper functions from the training script ---
from model_training import (
    add_features_to_single_batch, 
    scale_dataframe, 
    classifier_mapping,
    SEQ_LEN,
    FUTURE_PERIOD_PREDICT
)

# --- Load the pre-trained model and scalers ---
MODEL_PATH = 'path/to/your/model_checkpoint.keras'
SCALER_PATH = 'path/to/your/saved_scalers.joblib'

model = load_model(MODEL_PATH)
scalers = joblib.load(SCALER_PATH)

print("Model and scalers loaded successfully.")
model.summary()

### 2. Data Loading

Load the out-of-sample dataset. This data was not used during the training or validation phases to ensure the backtest provides an unbiased estimate of performance.

In [ ]:
DATA_PATH = 'data/market/unseen_data.parquet'
test_df = pd.read_parquet(DATA_PATH)

# For demonstration, we'll use a small slice of the data
backtest_data = test_df.head(1000).copy()

print(f"Loaded backtest data with shape: {backtest_data.shape}")

### 3. Simulation Loop & Inference

This is the core of the backtest. We iterate through each minute, simulate the flow of real-time data, and make trading decisions.

In [ ]:
# --- Backtest Configuration ---

trade_instrument = 'GBPUSD' # The instrument we are trading
trade_threshold = 0.50 # Confidence threshold to place a trade
minute_buffer = deque(maxlen=SEQ_LEN) # Buffer to hold recent data for model input
pnl_history = [] # List to store results of each closed trade

active_position = None # To hold info about the current open trade

# --- Main Simulation Loop ---
for timestamp, row in backtest_data.iterrows():
    minute_buffer.append(row)

    # We need a full buffer to make a prediction
    if len(minute_buffer) < SEQ_LEN:
        continue

    # 1. Prepare the data batch for the model
    input_batch_df = pd.DataFrame(list(minute_buffer))
    features_df = add_features_to_single_batch(input_batch_df.copy())
    
    # Drop rows with NaNs from feature calculation warm-up
    features_df.dropna(inplace=True)
    if len(features_df) < (SEQ_LEN - FUTURE_PERIOD_PREDICT):
        continue # Not enough data after processing

    # 2. Scale the data using the pre-fitted scalers
    scaled_df, _ = scale_dataframe(features_df.copy(), scalers, 'validation')
    model_input = scaled_df.drop(columns=['target']).values

    # 3. Generate a prediction
    prediction = model.predict(np.expand_dims(model_input, axis=0), verbose=0)[0]
    predicted_class = np.argmax(prediction)
    predicted_confidence = prediction[predicted_class]

    # --- Trade Execution Logic ---
    
    # 4. Check if we should close an existing position
    if active_position and (timestamp - active_position['entry_time']).total_seconds() >= (FUTURE_PERIOD_PREDICT * 60):
        if active_position['direction'] == 'LONG':
            exit_price = row[f'{trade_instrument}_close_price__bid_level_1'] # Sell at the bid
            pnl = exit_price - active_position['entry_price']
        else: # SHORT
            exit_price = row[f'{trade_instrument}_close_price__ask_level_1'] # Buy back at the ask
            pnl = active_position['entry_price'] - exit_price
        
        pnl_history.append({'pnl': pnl, 'exit_time': timestamp})
        active_position = None # Position is now closed

    # 5. Check if we should open a new position
    if not active_position and predicted_confidence > trade_threshold:
        direction = None
        if classifier_mapping[predicted_class] in ['strong upward', 'mild upward']:
            direction = 'LONG'
            entry_price = row[f'{trade_instrument}_open_price__ask_level_1'] # Buy at the ask
        elif classifier_mapping[predicted_class] in ['strong downward', 'mild downward']:
            direction = 'SHORT'
            entry_price = row[f'{trade_instrument}_open_price__bid_level_1'] # Sell at the bid
        
        if direction:
            active_position = {
                'entry_time': timestamp,
                'entry_price': entry_price,
                'direction': direction
            }

print("Backtest simulation complete.")

### 4. Performance Analysis

With the simulation finished, we can now analyze the results to understand the strategy's performance.

In [ ]:
if not pnl_history:
    print("No trades were executed during the backtest.")
else:
    results_df = pd.DataFrame(pnl_history)
    results_df.set_index('exit_time', inplace=True)

    # Calculate PnL in pips (assuming non-JPY pair)
    results_df['pnl_pips'] = results_df['pnl'] * 10000
    results_df['cumulative_pnl_pips'] = results_df['pnl_pips'].cumsum()

    # --- Print Key Metrics ---
    total_trades = len(results_df)
    winning_trades = (results_df['pnl_pips'] > 0).sum()
    win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
    total_pnl = results_df['pnl_pips'].sum()

    print(f"--- Backtest Results ---")
    print(f"Total Trades: {total_trades}")
    print(f"Win Rate: {win_rate:.2f}%")
    print(f"Total PnL: {total_pnl:.2f} pips")

    # --- Plot Cumulative PnL ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(14, 7))
    results_df['cumulative_pnl_pips'].plot(ax=ax, lw=2)

    ax.set_title('Cumulative PnL Over Time', fontsize=16)
    ax.set_xlabel('Date')
    ax.set_ylabel('Cumulative PnL (pips)')
    ax.axhline(0, color='black', linestyle='--', lw=1)

    plt.show()